In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import arviz as az
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import theano
print(theano.config.device)

np.random.seed(42)
pm.set_tt_rng(42)


cpu


In [363]:
C = 32
J = 2
K = 5
N = np.array([1000, 1000, 100, 30])
S = len(N)

# Hyper-parameter for priors
psi = np.ones(J)
gamma = np.ones(K) * 0.1
alpha = np.ones(C) * 0.1
beta = np.ones((4,K)) * 0.1
beta = np.repeat(np.array([[1, 0.1, 1, 1]]), K, axis=0)

In [364]:
# Generate data
theta_gen = pm.Dirichlet.dist(a=psi).random(size = S)
phi_gen = pm.Dirichlet.dist(a=alpha, shape=(C)).random(size = J)
A_gen = pm.Dirichlet.dist(a=gamma, shape=(J, K)).random(size = S)
# ACGT
# 0123
eta_gen = np.vstack([pm.Dirichlet.dist(a=beta[:,[0,2,3]]).random(size=C//2), 
                     pm.Dirichlet.dist(a=beta[:,[0,1,2]]).random(size=C//2)])

In [397]:
W=(theta_gen@phi_gen).T
Q=np.einsum('ij,ijk->ik', theta_gen, A_gen)@eta_gen
B=np.einsum('ij,ijk->jik',W,Q).reshape(S, -1)
X=np.vstack([d.random(size = 1) for d in map(pm.Multinomial.dist, N, B)])

## Full two step model

def encode_counts(X):
    # turn counts of position into word-style encoding
    # https://laptrinhx.com/topic-modeling-with-pymc3-398251916/
    S, C = X.shape
    return [np.repeat(range(C), X[i].astype(int)) for i in range(S)]

with pm.Model() as full_model:

    phi = pm.Dirichlet('phi', a=alpha, shape=(J, C))
    theta = pm.Dirichlet("theta", a=psi, shape=(S, J))
    A = pm.Dirichlet("A", a=gamma, shape = (S, J))
    etaC = pm.Dirichlet("etaC", a=beta[[0,2,3]], shape=(C, K))
    etaT = pm.Dirichlet("etaT", a=beta[[0,1,3]], shape=(C, K))
    eta = [etaC, etaT]
    
    for s in range(S):
        # draw damage context signature
        z = pm.Categorical(f"z_{i}", p=theta[s], shape=X[s].shape)
        # draw mutation damage context
        x = pm.Categorical(f"x_{i}", p = phi[z], shape=X[s].shape)
        # draw mutation signature
        v = pm.Categorical(f"v_{i}", p=A[z], shape=X[s].shape)
        # draw mutation
        y = pm.Categorical(f"y_{i}", p = eta[x > 16])

Fit the model

with full_model:    
    full_trace = pm.sample(1000, return_inferencedata=True)

Sanity check the result

az.plot_posterior(full_trace, var_names=['theta']);

plt.figure(figsize=(20,5))
for j in range(J):
    plt.subplot(2,1,j+1)
    for c in range(C):
        sns.distplot(full_trace.posterior['phi'][:, :, j, c], kde=False, hist=True, label=mut_types[c])
    plt.title(f'Phi density estimates for topic {j}')
    plt.legend(mut_types)

with full_model:
    phi_map = pm.find_MAP()['phi']
    print(np.diag(cosine_similarity(phi_map, phi_gen)))


## 'collapsed' two step LDA

In [ ]:
## Generate data
#theta_gen = pm.Dirichlet.dist(a=alpha).random(size = S)
#phi_gen = pm.Dirichlet.dist(a=beta, shape=(J, C)).random(size = 1).squeeze()
#X = np.vstack([d.random(size = 1) for d in map(pm.Multinomial.dist, N, theta_gen@phi_gen)])

# Generate data
theta_gen = np.array([0.3, 0.7, 0.8, 0.2, .5, .5]).reshape(S, J)
phi_gen = np.array([0.25, 0.6, 0.05, 0.05, 0, 0.05, 1,0,0,0,0,0]).reshape(J, C)
X = np.vstack([d.random(size = 1) for d in map(pm.Multinomial.dist, N, theta_gen@phi_gen)])

with pm.Model() as collapsed_model:

    phi = pm.Dirichlet('phi', a=beta, shape=(J, C))
    theta = pm.Dirichlet("theta", a=alpha, shape=(S, J))
    
    for i in range(S):
        # Words of document
        w = pm.Multinomial(f'sample_{i}', n = N[i], p = theta[i]@phi, observed=X[i])


Fit the model

with collapsed_model:
    collapsed_trace = pm.sample(1000, return_inferencedata=True)

#pm.plot_trace(trace);
#az.plot_forest(collapsed_trace, r_hat=True);
az.plot_posterior(collapsed_trace, var_names=['theta']);

plt.figure(figsize=(20,5))
for j in range(J):
    plt.subplot(2,1,j+1)
    for c in range(C):
        sns.distplot(collapsed_trace.posterior['phi'][:, :, j, c], kde=False, hist=True, label=mut_types[c])
    plt.title(f'Phi density estimates for topic {j}')
    plt.legend(mut_types)

with collapsed_model:
    phi_map = pm.find_MAP()['phi']
    print(np.diag(cosine_similarity(phi_map, phi_gen)))